In [2]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

In [33]:
%%sql
drop table if exists ArrayTbl;
CREATE TABLE ArrayTbl
 (keycol CHAR(1) PRIMARY KEY,
  col1  INTEGER,
  col2  INTEGER,
  col3  INTEGER,
  col4  INTEGER,
  col5  INTEGER,
  col6  INTEGER,
  col7  INTEGER,
  col8  INTEGER,
  col9  INTEGER,
  col10 INTEGER);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.


[]

In [34]:
%%sql
delete from ArrayTbl;
INSERT INTO ArrayTbl VALUES('A', NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL);
INSERT INTO ArrayTbl VALUES('B', 3, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL);

INSERT INTO ArrayTbl VALUES('D', NULL, NULL, 9, NULL, NULL, NULL, NULL, NULL, NULL, NULL);
INSERT INTO ArrayTbl VALUES('E', NULL, NULL, 9, NULL, NULL, 9, NULL, NULL, NULL, NULL);
INSERT INTO ArrayTbl VALUES('F', NULL, 3, NULL, 1, 9, NULL, NULL, 9, NULL, NULL);

*  postgresql://padawan:***@db:5432/dsdojo_db
0 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [ ]:
## やりたいこと
+ すべてnullであるものを見つける
+ 少なくとも一つがnot nullであるものを見つける
+ nつだけnot nullであるものを見つける

In [29]:
%%sql
-- すべてnullであるものを見つける
select *
from ArrayTbl
where coalesce(col1,col2,col3,col4,col5,col6,col7,col8,col9,col10) is null

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


keycol,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10
A,None,None,None,None,None,None,None,None,None,None


In [30]:
%%sql
-- すくなくとも1つがnot null
select *
from ArrayTbl
where coalesce(col1,col2,col3,col4,col5,col6,col7,col8,col9,col10) is not null

*  postgresql://padawan:***@db:5432/dsdojo_db
4 rows affected.


keycol,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10
B,3,None,None,None,None,None,None,None,None,None
D,None,None,9,None,None,None,None,None,None,None
E,None,None,9,None,None,9,None,None,None,None
F,None,3,None,1,9,None,None,9,None,None


In [36]:
%%sql
-- 1つだけnot nullであるものを見つける
select keycol
from (
    select keycol, col1, 'col1'
    from ArrayTbl
    union all
    select keycol, col2, 'col2'
    from ArrayTbl
    union all
    select keycol, col3, 'col3'
    from ArrayTbl
    union all
    select keycol, col4, 'col4'
    from ArrayTbl
    union all
    select keycol, col5, 'col5'
    from ArrayTbl
    union all
    select keycol, col6, 'col6'
    from ArrayTbl
    union all
    select keycol, col7, 'col7'
    from ArrayTbl
    union all
    select keycol, col8, 'col8'
    from ArrayTbl
    union all
    select keycol, col9, 'col9'
    from ArrayTbl
    union all
    select keycol, col10, 'col10'
    from ArrayTbl
) as tbl(keycol, col, label)
group by keycol
having count(case when col is not null then 1 else null end) = 1

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


keycol
B
D


In [31]:
%%sql
delete from ArrayTbl;
INSERT INTO ArrayTbl VALUES('A', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0);
INSERT INTO ArrayTbl VALUES('B', 3, 0, 0, 0, 0, 0, 0, 0, 0, 0);

INSERT INTO ArrayTbl VALUES('D', 0, 0, 9, 0, 0, 0, 0, 0, 0, 0);
INSERT INTO ArrayTbl VALUES('E', 0, 0, 9, 0, 0, 9, 0, 0, 0, 0);
INSERT INTO ArrayTbl VALUES('F', 0, 3, 0, 1, 9, 0, 0, 9, 0, 0);
-- すべて0であるものを見つける
select keycol
from ArrayTbl
where 0 = all(array[col1,col2,col3,col4,col5,col6,col7,col8,col9,col10]);

*  postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


keycol
A


In [22]:
%%sql
-- すくなくとも1つがnot null
select keycol
from ArrayTbl
where 0 <> any(array[col1,col2,col3,col4,col5,col6,col7,col8,col9,col10]);

*  postgresql://padawan:***@db:5432/dsdojo_db
4 rows affected.


keycol
B
D
E
F


In [19]:
%%sql
-- 1つのみが0
select keycol
from ArrayTbl
where sign(abs(col1))+
    sign(abs(col2))+
    sign(abs(col3))+
    sign(abs(col4))+
    sign(abs(col5))+
    sign(abs(col6))+
    sign(abs(col7))+
    sign(abs(col8))+
    sign(abs(col9))+
    sign(abs(col10)) = 1

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


keycol
B
D
